### 4. Criar um mapa das estações


Desafio(s):

- ler (parse) o cabeçalho para extrair os metadados em forma de tabela

Ferramenta(s):

- `open`, `for-loops`, `dictionary comprehension`
- [`cartopy`](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html)

In [ ]:
import pandas as pd


def parse_header(fname, start, end):
    copy = False

    with open(fname) as f:
        for line in f.readlines():
            line = line.strip()
            if start in line:
                bucket = [line.strip(start).strip()]
                copy = True
            elif end in line:
                copy = False
            elif copy:
                bucket.append(line)
    dic = {
        line.split("*")[0].strip(): {
            entry.split(":")[0].strip(): entry.split(":")[1].strip()
            for entry in line.split("*")[1:]
        }
        for line in bucket
    }

    return pd.DataFrame.from_dict(dic).T

In [ ]:
fname = "Habitats_sediments_Alkenone-GDGT-temperature_proxy.tab"
start = "Event(s):"
end = "Parameter(s):"

df = parse_header(fname, start, end).reset_index()

df.head()

In [ ]:
Event = [k[0].strip() for k in df["index"].str.split()]

df["Event"] = Event

In [ ]:
df.dtypes

In [ ]:
df["LATITUDE"] = pd.to_numeric(df["LATITUDE"])
df["LONGITUDE"] = pd.to_numeric(df["LONGITUDE"])
df["ELEVATION"] = pd.to_numeric(df["ELEVATION"].str.strip("m"))
df.dtypes

In [ ]:
df.to_csv("stations.csv", index=False)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.feature import NaturalEarthFeature
from oceans.datasets import etopo_subset

# Bounding box with some spacing to spare.
dx = dy = 1.5
bbox = (
    df["LONGITUDE"].min() - dx,
    df["LONGITUDE"].max() + dx,
    df["LATITUDE"].min() - dy,
    df["LATITUDE"].max() + dy,
)


# Coastline
feature = NaturalEarthFeature(
    name="coastline",
    category="physical",
    scale="10m",
    edgecolor="#000000",
    facecolor="#AAAAAA",
)

# Bathymetry
x, y, topo = etopo_subset(*bbox, smoo=True)
levels = [-3000, -2000, -1000, -500, -150, -100, -25]

# Figure
fig, ax = plt.subplots(
    figsize=(9, 9),
    subplot_kw={
        "projection": ccrs.PlateCarree(),
    },
)
ax.set_extent(bbox)

ax.add_feature(feature, zorder=0)
ax.scatter(
    df["LONGITUDE"], df["LATITUDE"], facecolor="none", edgecolor="black", zorder=1
)


def fmt(x):
    s = f"{x:.1f}"
    return rf"{s} m"


manual = [
    (-43, -23),
    (-43, -23.2),
    (-43, -23.8),
    (-42.9, -24),
    (-42.6, -24.4),
    (-42.5, -25),
    (-40.5, -25),
]
cs = ax.contour(
    x, y, topo, levels=levels, colors="black", alpha=0.25, zorder=0, linestyles="-"
)
ax.clabel(cs, cs.levels, inline=True, fmt=fmt, fontsize=10, manual=manual)

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xlines = gl.ylines = False

In [ ]:
import folium


m = folium.Map()


for k, row in df.iterrows():
    location = row["LATITUDE"], row["LONGITUDE"]
    folium.Marker(location=location, popup=f"{row['index']}").add_to(m)

m.fit_bounds(m.get_bounds())

m